# Primo caso

- pip install scikit-learn
- pip install sentence-transformers
- pip install langdetect

In [1]:
# Libreria che consente la rappresentazione vettoriale (embedded) di frasi anziché parole
from sentence_transformers import SentenceTransformer
# Libreria per calcolare la similarità coseno tra vettori (la similiarità coseno misura quanto due vettori sono simili)
from sklearn.metrics.pairwise import cosine_similarity
# Libreria per il rilevamento della lingua del testo
from langdetect import detect

# Carica modello multilingua potente che comprende l'italiano
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Lista statica di API con descrizioni ed endpoint
api_catalog = [
    {
        "descrizione": "Creazione nuovo finanziamento. Creazione per tipo e ndg",
        "endpoint": "api/finanziamento",
        "verbo": "POST",
    },
    {
        "descrizione": "Ricerca un finanziamento esistente. Ricerca per tipo e ndg",
        "endpoint": "api/finanziamento&tipo=mutuo&ndg=123456",
        "verbo": "GET",
    }
]

def get_api(testo_input, soglia_similarita=0.5):
    try:
        # Verifica che la lingua sia italiana
        if detect(testo_input) != "it":
            return "Per favore fornisci il testo in italiano."

        # Calcola embedding del testo utente
        embedding_input = model.encode([testo_input])

        migliori_match = {"endpoint": None, "score": 0.0}

        # Confronta con le descrizioni delle API
        for api in api_catalog:
            # Calcola embedding della descrizione dell'API
            embedding_descrizione = model.encode([api["descrizione"]])
            # Calcola la similarità coseno tra l'input e la descrizione dell'API
            sim = cosine_similarity(embedding_input, embedding_descrizione)[0][0]

            # Aggiorna il miglior match se la similarità è maggiore della soglia
            if sim > migliori_match["score"]:
                migliori_match = {"endpoint": api["endpoint"], "score": sim}

        # Controlla se il miglior match supera la soglia di similarità
        if migliori_match["score"] >= soglia_similarita:
            return f"{migliori_match['endpoint']}"
        else:
            return "La richiesta non trova corrispondenza con nessuna API. Riprova."

    except Exception as e:
        return f"Errore durante l'elaborazione: {str(e)}"


C:\Users\edoardo.frapiccini\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Test primo caso

In [2]:
esempi_richiesta = [
    "cercami i finanziamenti dell'ndg 123456",
    "voglio avviare un nuovo finanziamento",
    "mi cerchi un finanziamento",
    "ho bisogno di un mutuo per la casa",
    "vado a comprarmi una pizza",
    "mi crei un finanziamento con ndg 123456 e tipo mutuo",
]

for richiesta in esempi_richiesta:
    print(f"  Richiesta utente: {richiesta}")
    print(f"Endpoint associato: {get_api(richiesta)}\n")

  Richiesta utente: cercami i finanziamenti dell'ndg 123456
Endpoint associato: api/finanziamento&tipo=mutuo&ndg=123456

  Richiesta utente: voglio avviare un nuovo finanziamento
Endpoint associato: api/finanziamento

  Richiesta utente: mi cerchi un finanziamento
Endpoint associato: api/finanziamento&tipo=mutuo&ndg=123456

  Richiesta utente: ho bisogno di un mutuo per la casa
Endpoint associato: La richiesta non trova corrispondenza con nessuna API. Riprova.

  Richiesta utente: vado a comprarmi una pizza
Endpoint associato: La richiesta non trova corrispondenza con nessuna API. Riprova.

  Richiesta utente: mi crei un finanziamento con ndg 123456 e tipo mutuo
Endpoint associato: api/finanziamento



# Secondo caso

- pip install spacy
- python -m spacy download it_core_news_md

# Importo librerie

In [3]:
# Libreria per l'elaborazione del linguaggio naturale
import spacy
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Addestramento machine learning 

In [4]:
X_train = [
    # ==== GET ====
    "cerca", "cercami", "fai una ricerca", "esegui una ricerca",
    "trova", "trovami", "rintraccia", "identifica",
    "recupera", "recuperami", "ottieni dati", "estrai informazioni",
    "mostra", "mostrami", "fammi vedere", "visualizza",
    "leggi", "leggimi", "accedi ai dati", "ottieni i dati",
    "visualizza", "visualizzami", "rendi visibile", "presenta",
    "vedi", "vedimi", "guarda i dati", "dammi una vista",
    "estrai", "estraimi", "scarica dati", "porta fuori dati",
    "accedi", "accedimi", "entra nei dati", "consulta",

    # ==== POST ====
    "crea", "creami", "genera nuovo", "costruisci un nuovo",
    "inserisci", "inseriscimi", "aggiungi", "carica nuovi dati",
    "richiedi", "richiedimi", "fai una richiesta", "manda una richiesta",
    "apri", "aprimi", "avvia una nuova pratica", "inizia procedura",
    "avvia", "avviami", "dai inizio", "comincia processo",
    "registra", "registrami", "salva nuovo", "archivia dati",
    "attiva", "attivami", "metti in funzione", "abilita",
    "compila", "compilami", "riempi i dati", "completa il modulo",

    # ==== PUT ====
    "aggiorna", "aggiornami", "fai un aggiornamento", "modifica con nuovi dati",
    "modifica", "modificami", "cambia i dati", "rivedi i valori",
    "correggi", "correggimi", "sistema dati", "risolvi errori",
    "rivedi", "rivedimi", "verifica e modifica", "ritocca",
    "sostituisci", "sostituiscimi", "cambia con altro", "scambia contenuto",
    "ricalcola", "ricalcolami", "rifai i conti", "esegui nuovo calcolo",

    # ==== DELETE ====
    "elimina", "eliminami", "cancella definitivamente", "rimuovi per sempre",
    "cancella", "cancellami", "butta via", "togli dai dati",
    "rimuovi", "rimuovimi", "escludi", "levami dai dati",
    "revoca", "revocami", "invalida", "annulla autorizzazione",
    "annulla", "annullami", "ferma l’operazione", "interrompi processo",
    "disattiva", "disattivami", "spegni", "rendi inattivo"
]

y_train = [
    # ==== GET ====
    "GET", "GET", "GET", "GET", 
    "GET", "GET", "GET", "GET", 
    "GET", "GET", "GET", "GET",       
    "GET", "GET", "GET", "GET", 
    "GET", "GET", "GET", "GET", 
    "GET", "GET", "GET", "GET", 
    "GET", "GET", "GET", "GET",       
    "GET", "GET", "GET", "GET", 
    "GET", "GET", "GET", "GET", 
    # ==== POST ====
    "POST", "POST", "POST", "POST",
    "POST", "POST", "POST", "POST",
    "POST", "POST", "POST", "POST",
    "POST", "POST", "POST", "POST",
    "POST", "POST", "POST", "POST",
    "POST", "POST", "POST", "POST",
    "POST", "POST", "POST", "POST",
    "POST", "POST", "POST", "POST",
    # ==== PUT ====
    "PUT", "PUT", "PUT", "PUT",
    "PUT", "PUT", "PUT", "PUT",
    "PUT", "PUT", "PUT", "PUT",
    "PUT", "PUT", "PUT", "PUT",
    "PUT", "PUT", "PUT", "PUT",
    "PUT", "PUT", "PUT", "PUT",
    # ==== DELETE ====
    "DELETE", "DELETE", "DELETE", "DELETE",
    "DELETE", "DELETE", "DELETE", "DELETE",
    "DELETE", "DELETE", "DELETE", "DELETE",
    "DELETE", "DELETE", "DELETE", "DELETE",
    "DELETE", "DELETE", "DELETE", "DELETE",
    "DELETE", "DELETE", "DELETE", "DELETE",
]

# Carica il modello italiano
nlp = spacy.load("it_core_news_md")

#metodo per trasformare x_train che contiene frasi in vettori numerici
def vectorizza(frasi):
    return np.array([nlp(frase).vector for frase in frasi])

# Vettorizza il training set
X_vect = vectorizza(X_train)

# Definisci il classificatore
clf = LogisticRegression()

# Allena il classificatore
clf.fit(X_vect, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


# Utils

In [ ]:
# Metodo per analizzare il testo e suddividerlo in frasi
""" Il metodo riconosce più frasi separate da punteggiatura, invece, ad esempio, non riconosce invece frasi congiunte da "e" o "poi" """
def analizza_frasi(testo: str):
    # Analizza la frase con spaCy
    doc = nlp(testo)
    # Estrai le frasi dal testo
    frasi = [sent.text.strip() for sent in doc.sents] 
    print(f"       Frasi estratte: {frasi}")
    return len(frasi), frasi

# Metodo per estrarre verbo e complemento oggetto da una frase
def estrai_verbo_oggetto(frase: str, include_oggetto: bool = False):
    # Analizza la frase con spaCy
    doc = nlp(frase)
    risultati = []
    # Itera sui token della frase
    for token in doc:
        # Verbi
        if token.pos_ == "VERB":
            # Aggiungi il verbo alla lista dei risultati
            risultati.append(("VERBO", token.text))
        # Complemento oggetto (solo se richiesto dal flag)
        if include_oggetto and token.dep_ == "obj":
            # Aggiungi l'oggetto alla lista dei risultati
            risultati.append(("OGGETTO", token.text))
    return risultati

def classifica_verbo(testo_input: str, include_oggetto: bool = False):
    # Estraggo le caratteristiche del verbo (e oggetto se richiesto)
    caratteristiche_verbo = estrai_verbo_oggetto(testo_input, include_oggetto)
    # Estraggo solo la parola del verbo (escludendo etichetta e posizioni)
    verbo = [v[1] for v in caratteristiche_verbo if v[0] == "VERBO"]
    # Vettorizzazione
    verbo_vect = vectorizza(verbo)
    # Predizione
    verbo_predetto = clf.predict(verbo_vect)[0]
    # Probabilità per tutte le classi
    probs = clf.predict_proba(verbo_vect)[0]   # array 1D tipo [0.05, 0.10, 0.82, 0.03]
    # Indice della classe predetta
    verbo_predetto_indice = clf.classes_.tolist().index(verbo_predetto)
    # Probabilità della classe predetta
    score_verbo_predetto = probs[verbo_predetto_indice]
    print("--- ML")
    print(f"       Verbo estratto: {verbo}")
    print(f"  Verbo http predetto: {verbo_predetto}")
    print(f"               Classi: {clf.classes_}")
    print(f"          Probabilità: {probs}")
    print(f"  Indice della classe: {verbo_predetto_indice}")
    print(f"Probabilità del verbo: {score_verbo_predetto}")
    #esempio
    #   Verbo http predetto       → 'POST'
    #   Classi                    → ['DELETE', 'GET', 'POST', 'PUT']
    #   Probabilità               → [0.05, 0.10, 0.82, 0.03]  # stesso ordine
    #   Indice della classe       → 2  (perché 'POST' è il 3° elemento)
    #   Probabilità del verbo     → verbo_probabilita[2] = 0.82
    return probs



# Api da individuare

In [6]:
# Lista statica di API con descrizioni ed endpoint
api_catalog = [
    {
        "descrizione": "Creazione nuovo finanziamento. Creazione per tipo e ndg",
        "endpoint": "api/finanziamento",
        "verbo_http": "POST"
    },
    {
        "descrizione": "Ricerca un finanziamento esistente. Ricerca per tipo e ndg",
        "endpoint": "api/finanziamento?tipo=mutuo&ndg=123456",
        "verbo_http": "GET"
    },
    {
        "descrizione": "Cancellazione finanziamento esistente. Cancellazione per ndg",
        "endpoint": "api/finanziamento?ndg=123456",
        "verbo_http": "DELETE"
    },
    {
        "descrizione": "Aggiornamento finanziamento esistente. Aggiornamento per ndg",
        "endpoint": "api/finanziamento?ndg=123456",
        "verbo_http": "PUT"
    }
]

# Algoritmo

In [12]:
# Carica modello multilingua potente che comprende l'italiano
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def get_api_2(testo_input, include_oggetto=False, ml_weight=0.5, nlp_weight=0.5, soglia_similarita=0.5):
    try:
        print(f"     Richiesta utente: {testo_input}")

        # Verifica che la richiesta contenga una sola frase
        num_frasi, lista_frasi = analizza_frasi(testo_input)
        if num_frasi > 1:
            return f"                       La richiesta contiene {num_frasi} frasi. Per favore invia una sola frase alla volta. Frasi rilevate: {lista_frasi}"
        
        # Recupera le probabilità per ogni verbo HTTP
        probs = classifica_verbo(testo_input, include_oggetto)

        # Calcola embedding del testo utente
        embedding_input = model.encode([testo_input])

        migliore_match = {"endpoint": None, "verbo_http": None, "score": 0.0}
        migliore_match_composito = {"endpoint": None, "verbo_http": None, "score": 0.0}

        # Confronta con le descrizioni delle API
        for api in api_catalog:

            # Calcola embedding della descrizione dell'API
            embedding_api = model.encode([api["descrizione"]])

            # Calcola la similarità coseno tra l'input e la descrizione dell'API
            sim = cosine_similarity(embedding_input, embedding_api)[0][0]

            # Aggiorna il miglior match se la similarità è maggiore della soglia
            if sim > migliore_match["score"]:
                migliore_match = {"endpoint": api["endpoint"], "verbo_http": api["verbo_http"], "score": sim}

            
            # Combina i due score (similarità e probabilità verbo) con pesi alpha e beta
            verbo_indice = clf.classes_.tolist().index(api["verbo_http"])
            score_verbo = probs[verbo_indice]
            sim_composito = ml_weight * score_verbo + nlp_weight * sim
            if sim_composito > migliore_match_composito["score"]:
                migliore_match_composito = {"endpoint": api["endpoint"], "verbo_http": api["verbo_http"], "score": sim_composito}
            

        print("--- NLP")
        print(f"    Similarità coseno: {migliore_match}")
        print("--- ML + NLP")

        # Controlla se il miglior match supera la soglia di similarità
        if migliore_match_composito["score"] >= soglia_similarita:
            return f"                       {migliore_match_composito}"
        else:
            return "                       La richiesta non trova corrispondenza con nessuna API. (score: " + str(migliore_match_composito["score"]) + ")"

    except Exception as e:
        return f"Errore durante l'elaborazione: {str(e)}"


# Test secondo caso

In [ ]:
esempi_richiesta = [
    "cercami i finanziamenti dell'ndg 123456",
    "voglio avviare un nuovo finanziamento",
    "mi cerchi un finanziamento",
    "ho bisogno di un mutuo per la casa",
    "vado a comprarmi una pizza",
    "mi crei un finanziamento con ndg 123456 e tipo mutuo",
]

for richiesta in esempi_richiesta:
    print("----------------------------------------------------------------")
    print(get_api_2(richiesta, include_oggetto=False, ml_weight=0.1, nlp_weight=0.9, soglia_similarita=0.6)),
    print("----------------------------------------------------------------\n")

----------------------------------------------------------------
     Richiesta utente: cercami i finanziamenti dell'ndg 123456
       Frasi estratte: ["cercami i finanziamenti dell'ndg 123456"]
--- ML
       Verbo estratto: ['cercami']
  Verbo http predetto: GET
               Classi: ['DELETE' 'GET' 'POST' 'PUT']
          Probabilità: [9.66234521e-06 9.99464908e-01 5.23321003e-04 2.10856070e-06]
  Indice della classe: 1
Probabilità del verbo: 0.9994649080912499
--- NLP
    Similarità coseno: {'endpoint': 'api/finanziamento?tipo=mutuo&ndg=123456', 'verbo_http': 'GET', 'score': np.float32(0.71034634)}
--- ML + NLP
                       {'endpoint': 'api/finanziamento?tipo=mutuo&ndg=123456', 'verbo_http': 'GET', 'score': np.float64(0.739258162066144)}
----------------------------------------------------------------

----------------------------------------------------------------
     Richiesta utente: voglio avviare un nuovo finanziamento
       Frasi estratte: ['voglio avviare un nu

# TODO:
- Pensare a come gestire l'inserimento di più frasi da parte dell'utente in un'unica richiesta
- Migliorare la parte di ML, ad esempio: 
  - cercare una libreria che data una parola ti fornisce tutti i sinonimi possibili (o una cosa del genere), potrebbe essere utile per costruire un dataset di input in modo automatico
  - ci sono modelli migliore del LogisticRegression? 
  - trovare una parametrizzazione efficiente per addestrare il modello scelto
- Capire come migliorare "live" l'algoritmo... renforce learning?
- AI può suuggerire le prossime azioni da fare (dettate dal workflow)?